## Setup

In [1]:
import re
import nltk
import pickle
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
nltk.download(["stopwords"])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sahilshaheen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
eng_stopwords = stopwords.words("english")

### Loading train data

In [4]:
train_df = pd.read_csv("data/labeledTrainData.tsv", delimiter="\t", quoting=3)

In [5]:
train_df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [6]:
print(train_df["review"][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

## Preprocessing

### Cleaning reviews

In [7]:
def clean_review(review):
    review = BeautifulSoup(review).text
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', review)
    review = re.sub("[^a-zA-Z]", " ",review)
    review = review.lower().split()
    review = [w for w in review if not w in eng_stopwords]
    review = " ".join(review + emoticons)
    return review

In [8]:
cleaned_reviews = []

for review in train_df["review"]:
    cleaned_reviews.append(clean_review(review))

In [9]:
print(cleaned_reviews[0])

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

### Splitting and representing reviews using BoW

In [10]:
X_train, X_test, y_train, y_test = train_test_split(cleaned_reviews, train_df["sentiment"], test_size=0.2, random_state=0)

In [11]:
vectorizer = CountVectorizer()
# vectorizer = TfidfVectorizer()

In [12]:
train_bag = vectorizer.fit_transform(X_train)
test_bag = vectorizer.transform(X_test)

## Training Random Forest

In [13]:
rf = RandomForestClassifier(random_state=0)

In [14]:
rf.fit(train_bag, y_train)

RandomForestClassifier(random_state=0)

In [15]:
rf.score(test_bag, y_test)

0.8598

In [16]:
pickle.dump(rf, open('../model.pkl', 'wb'))

In [17]:
pickle.dump(vectorizer, open("../vect.pkl", "wb"))